In [1]:

from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# check model from the paper: https://github.com/zhengxu001/neural-data-augmentation

!pip install config
import os
import scipy.io
import scipy.misc
import numpy as np
import argparse
from keras.layers import Dense
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
import shutil
import itertools
import tensorflow as tf
import config


In [3]:
path = "/content/gdrive/MyDrive/CV-DL/"
def rename_Styles():
    data_path = path + "Styletransfer_results_2/"
    categories = os.listdir(data_path)
    print(categories)
    for cat in categories:
        data = data_path + str(cat) + "/"
        image_files = os.listdir(os.path.join(data_path, cat))
        # print(image_files)
        for file in image_files:
          old_name = file
          file =file.split(".")
          new_name = file[0] + "_style_2." + file[1]

          src = data+old_name
          dest = data+new_name
          os.rename(src, dest) 
          # print(src, dest)

# rename_Styles()


In [4]:
path = "/content/gdrive/MyDrive/CV-DL/"

import os
import shutil
import itertools
import numpy as np
import config
import argparse

def split_dataset():
    data_path = path + "101_ObjectCategories/"
    test_path = path + "101_ObjectCategories_test/"
    categories = os.listdir(data_path)
    print(categories)
    for cat in categories:
        image_files = os.listdir(os.path.join(data_path, cat))
        choices = np.random.choice([0, 1], size=(len(image_files),), p=[0.70, .30])
        files_to_move = itertools.compress(image_files, choices)
        
        for _f in files_to_move:
            print(_f)
            origin_path = os.path.join(data_path, cat, _f)
            dest_dir = os.path.join(test_path, cat)
            dest_path = os.path.join(test_path, cat, _f)
            if not os.path.isdir(dest_dir):
                os.makedirs(dest_dir)
            shutil.move(origin_path, dest_path)

#only do this to make the test dataset, commented out for now
# split_dataset()

In [5]:


def augmentation():
        train_datagen = ImageDataGenerator(
            rescale=1. / 255)
        val_datagen = ImageDataGenerator(
            rescale=1. / 255)
        return train_datagen, val_datagen

def get_datasets(train_path, test_path):        
        train_dataset = path + train_path
        validation_dataset = path + test_path
        class_number = 12
        return train_dataset, validation_dataset, class_number




In [ ]:
#datapaths to both datasets
data_dir_stylized_1 =   "/Styletransfer_results_1/"
data_dir_stylized_2 =   "/Styletransfer_results_2/"
data_dir_stylized_3 =   "/Styletransfer_results_3/"
data_dir_stylized_4 =   "/Styletransfer_results_all/"
data_path_1 =  "12_ObjectCategories/"
data_path_2 =  "101_ObjectCategories/"
data_dir_aug = "/5_augmentations_style_all/"

#fill this in!!!!
train_path = data_dir_aug

test_path=  "12_ObjectCategories_test/"


#for saving checkpoints
checkpoint_path = path + "models/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

aug_strategy="NA"
model = 'vgg16'
dataset = "caltech101"
num_epochs = 70
style ="NA" 



def build_vgg_models(model, dataset, epochs, aug_strategy, train_path, test_path):
    print("User Model", model)
    print("Expected Epochs is", epochs)

    train_dataset, validation_dataset, class_number = get_datasets(train_path, test_path)
      
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

    tbCallBack = TensorBoard(log_dir= "logs", histogram_freq=0, write_graph=True, write_images=True)

    #saves weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    vgg = eval(model)(weights='imagenet')
    fc2 = vgg.get_layer('fc2').output
    prediction = Dense(units=class_number, activation='softmax', name='logit')(fc2)
    model = Model(vgg.input, prediction)
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.0001), metrics=['accuracy'])

    model.summary()
    #only for normal augmentations for now
    train_datagen, val_datagen = augmentation()
      
    train_generator = train_datagen.flow_from_directory(
          train_dataset,
          target_size=(224,224),
          batch_size=32,
          class_mode='categorical')
    val_generator = train_datagen.flow_from_directory(
          validation_dataset,
          target_size=(224,224),
          batch_size=32,
          shuffle=False, 
          class_mode='categorical')

    model.fit(train_generator, validation_data=val_generator,
          epochs=epochs, steps_per_epoch = None, callbacks=[earlyStopping, tbCallBack])

build_vgg_models(model.upper(),dataset, num_epochs, aug_strategy, train_path, test_path)







User Model VGG16
Expected Epochs is 70
553476096/553467096 [==============================] - 19s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________

In [ ]:
# %load_ext tensorboard
import tensorflow as tf
import datetime, os

%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.
